In [1]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
import io
import sys
import logging
from new_strategy import TradingStrategy, Asset, BetSizingMethod, get_bet_sizing
from meta_strategy import MetaLabelingStrategy
import nbimporter
from backtest import Backtest
from add_features import generate_meta_labeled_data, merge_with_raw_features
from metalabel_backtest import MetaModelHandler
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
%load_ext autoreload
%autoreload 2

In [2]:
def split_price_data(price_data: pd.DataFrame, split_ratio: float = 0.7):
    split_idx = int(len(price_data) * split_ratio)
    return price_data.iloc[:split_idx], price_data.iloc[split_idx:]

def run_training_phase(asset: Asset, method: BetSizingMethod, price_data: pd.DataFrame, feature_cols: list):
    # Run base strategy
    past_returns = price_data['close'].pct_change().dropna()
    bet_sizing = get_bet_sizing(method, past_returns)
    
    strategy = TradingStrategy(price_data, asset.value, bet_sizing, method)
    strategy.generate_signals()
    strategy.simulate_trades()
    
    trade_df = strategy.get_trade_data()
    labeled = generate_meta_labeled_data(trade_df)
    #labeled_with_features = merge_with_raw_features(labeled, asset.value)
    labeled_with_features = labeled
    
    print(labeled_with_features.columns)
    return labeled_with_features

def train_meta_model(train_df: pd.DataFrame, feature_cols: list) -> MetaModelHandler:
    model = MetaModelHandler()
    model.train(train_df, feature_cols)
    return model

def run_parallel_evaluation(asset, method, test_price_data, signals, meta_model, feature_cols):
    past_returns = test_price_data['close'].pct_change().dropna()
    bet_sizing = get_bet_sizing(method, past_returns)

    #  First, initialize Meta strategy to access feature-enriched data
    meta_strategy = MetaLabelingStrategy(
        test_price_data.copy(), asset.value, bet_sizing, method,
        meta_model_handler=meta_model
    )
    feature_data = meta_strategy.data.copy()

    #  Use feature_data for both strategies
    baseline = TradingStrategy(feature_data.copy(), asset.value, bet_sizing, method)
    baseline.trade_signals = signals

    filtered = MetaLabelingStrategy(feature_data.copy(), asset.value, bet_sizing, method, meta_model_handler=meta_model)
    filtered.trade_signals = signals

    print("\n📊 Baseline features in .data:")
    print(baseline.data.columns.tolist())

    print("\n📊 Filtered features in .data:")
    print(filtered.data.columns.tolist())
    print("\n🕒 Baseline index:", baseline.data.index.name, baseline.data.index.dtype)
    print("🕒 Filtered index:", filtered.data.index.name, filtered.data.index.dtype)

    baseline.simulate_trades()
    filtered.simulate_trades()

    return baseline, filtered


def compare_backtests(baseline: TradingStrategy, filtered: MetaLabelingStrategy, asset: Asset, method: BetSizingMethod):
    print("\n[BASELINE]")
    baseline_bt = Backtest(baseline)
    baseline_bt.run_analysis()
    baseline_buffer = io.StringIO()
    sys.stdout = baseline_buffer
    baseline_bt.print_summary()
    sys.stdout = sys.__stdout__
    baseline_summary = baseline_buffer.getvalue()
    print(baseline_summary)

    print("\n[META-FILTERED]")
    filtered_bt = Backtest(filtered)
    filtered_bt.run_analysis()
    filtered_buffer = io.StringIO()
    sys.stdout = filtered_buffer
    filtered_bt.print_summary()
    sys.stdout = sys.__stdout__
    filtered_summary = filtered_buffer.getvalue()
    print(filtered_summary)

    print(f"\nTrades Rejected by Meta-Model: {filtered.rejected_trades}")

    output_dir = "results_metalabel"
    os.makedirs(output_dir, exist_ok=True)

    asset_name = asset.value.lower()
    method_name = method.value.lower()

    baseline_csv = f"baseline_{asset_name}_{method_name}.csv"
    filtered_csv = f"filtered_{asset_name}_{method_name}.csv"
    baseline_txt = f"baseline_{asset_name}_{method_name}.txt"
    filtered_txt = f"filtered_{asset_name}_{method_name}.txt"

    baseline.get_trade_data().to_csv(os.path.join(output_dir, baseline_csv), index=False)
    filtered.get_trade_data().to_csv(os.path.join(output_dir, filtered_csv), index=False)

    with open(os.path.join(output_dir, baseline_txt), "w") as f:
        f.write(baseline_summary)

    with open(os.path.join(output_dir, filtered_txt), "w") as f:
        f.write(filtered_summary)

    print(f"\n✅ CSVs saved to: {output_dir}")
    print(f" - {baseline_csv}")
    print(f" - {filtered_csv}")
    print(f"\n✅ Summaries saved to:")
    print(f" - {baseline_txt}")
    print(f" - {filtered_txt}")


if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)

    asset = Asset.XAUUSD
    method = BetSizingMethod.OPTIMAL_F

    feature_cols = ["attempt", "ma_14", "min_price_30", "max_price_30","atr_14"]
    price_path = Path(f"data/processed/{asset.value}/combined_data.csv")
    price_data = pd.read_csv(price_path, index_col='timestamp', parse_dates=True)

    # Split
    train_data, test_data = split_price_data(price_data)

    # Training
    train_trades = run_training_phase(asset, method, train_data, feature_cols)
    train_trades['session_code'] = train_trades['session'].map({
    'asian': 0,
    'london': 1,
    'us': 2
    })


    clean_features = train_trades[feature_cols].replace([np.inf, -np.inf], np.nan).dropna()
    train_trades_cleaned = train_trades.loc[clean_features.index]
    meta_model = train_meta_model(train_trades_cleaned, feature_cols)

    # Reuse signals
    signal_gen = TradingStrategy(test_data.copy(), asset.value, get_bet_sizing(method), method)
    signal_gen.generate_signals()
    shared_signals = signal_gen.trade_signals

    print("\n🔍 Entry Time Check (from shared_signals dict):")
    print("Type:", type(shared_signals))
    print("Keys in shared_signals:", list(shared_signals.keys()))

    # Try to peek at the first signal (depending on structure)
    first_key = next(iter(shared_signals))
    first_signal_list = shared_signals[first_key]
    first_signal = first_signal_list[0]  # Get the first signal from the list
    print("Type of entry_time:", type(first_signal['entry_time']))

    # Evaluation
    base_strat, filtered_strat = run_parallel_evaluation(
        asset, method, test_data, shared_signals, meta_model, feature_cols
    )

    # Compare
    compare_backtests(base_strat, filtered_strat, asset, method)

2025-06-11 14:31:49,871 - INFO - Strategy initialized for XAUUSD using OptimalF


[OptimalF] Session: asian, available trades: 0/20
[OptimalF] (Default) session=asian, equity=100000.00, price=1518.71, risk=0.0100, risk_amount=1000.00
📦 Creating TradeSetup with:
  entry_time: 2020-01-02 00:00:00+00:00
  atr_14: nan, ma_14: nan, min_price_30: nan, max_price_30: nan
  context keys: ['atr_14', 'ma_14', 'min_price_30', 'max_price_30', 'attempt', 'ref_close', 'session']
[OptimalF] Trade added: session=asian, pnl=0.17, total_trades=1
[OptimalF] Session: london, available trades: 0/20
[OptimalF] (Default) session=london, equity=100000.00, price=1518.97, risk=0.0100, risk_amount=1000.00
📦 Creating TradeSetup with:
  entry_time: 2020-01-02 08:00:00+00:00
  atr_14: nan, ma_14: nan, min_price_30: nan, max_price_30: nan
  context keys: ['atr_14', 'ma_14', 'min_price_30', 'max_price_30', 'attempt', 'ref_close', 'session']
[OptimalF] Trade added: session=london, pnl=5.00, total_trades=1
[OptimalF] Session: us, available trades: 0/20
[OptimalF] (Default) session=us, equity=100000.0

2025-06-11 14:36:15,638 - INFO - Strategy initialized for XAUUSD using OptimalF
2025-06-11 14:36:59,938 - INFO - Strategy initialized for XAUUSD using OptimalF



🔍 Entry Time Check (from shared_signals dict):
Type: <class 'dict'>
Keys in shared_signals: ['asian', 'london', 'us']
Type of entry_time: <class 'pandas._libs.tslibs.timestamps.Timestamp'>


2025-06-11 14:37:00,144 - INFO - Strategy initialized for XAUUSD using OptimalF
2025-06-11 14:37:00,199 - INFO - Strategy initialized for XAUUSD using OptimalF



📊 Baseline features in .data:
['Gmt time', 'open', 'high', 'low', 'close', 'volume', 'date', 'time', 'asian_session', 'london_session', 'us_session', 'prev_close', 'daily_high', 'daily_low', 'daily_close', 'true_range', 'atr_14', 'ma_14', 'ma_30', 'ma_100', 'day_of_week', 'week_number', 'max_price_14', 'min_price_14', 'max_price_30', 'min_price_30', 'max_price_100', 'min_price_100', 'drawdown_static', 'drawdown_30']

📊 Filtered features in .data:
['Gmt time', 'open', 'high', 'low', 'close', 'volume', 'date', 'time', 'asian_session', 'london_session', 'us_session', 'prev_close', 'daily_high', 'daily_low', 'daily_close', 'true_range', 'atr_14', 'ma_14', 'ma_30', 'ma_100', 'day_of_week', 'week_number', 'max_price_14', 'min_price_14', 'max_price_30', 'min_price_30', 'max_price_100', 'min_price_100', 'drawdown_static', 'drawdown_30']

🕒 Baseline index: timestamp datetime64[ns, UTC]
🕒 Filtered index: timestamp datetime64[ns, UTC]
[OptimalF] Session: asian, available trades: 0/20
[OptimalF] 

2025-06-11 14:37:27,547 - INFO - Exported detailed trades to data/results/trades_detailed_XAUUSD_optimal_f.csv
2025-06-11 14:37:27,608 - INFO - Saved backtest results to data/results/backtest_results_XAUUSD_optimal_f.txt



=== XAUUSD Backtest Results ===
Bet Sizing Strategy: optimalf

Period: 2023-06-08 to 2024-11-22

ASIAN Session Performance:
Initial Capital: $100,000.00
Final Capital: $100,169.38
Total PnL: $169.38
Return: 0.17%
Win Rate: 49.54% (214W/218L)
Max Drawdown: 0.91%
Sharpe Ratio: 0.16
Skewness: 1.343
Excess Kurtosis: 38.098

Attempt Analysis:

  Attempt 1:
  Trades: 382 (88.4% of session trades)
  PnL: $186.87
  Win Rate: 50.00% (191W/191L)
  Average Win: $14.97
  Average Loss: $-14.21

  Attempt 2:
  Trades: 45 (10.4% of session trades)
  PnL: $-0.24
  Win Rate: 51.11% (23W/22L)
  Average Win: $1.82
  Average Loss: $-1.91

  Attempt 3:
  Trades: 5 (1.2% of session trades)
  PnL: $-17.25
  Win Rate: 0.00% (0W/5L)
  Average Win: $0.00
  Average Loss: $-3.45

LONDON Session Performance:
Initial Capital: $100,000.00
Final Capital: $102,368.73
Total PnL: $2,368.73
Return: 2.37%
Win Rate: 50.19% (264W/262L)
Max Drawdown: 0.99%
Sharpe Ratio: 0.83
Skewness: 0.022
Excess Kurtosis: 6.331

Attempt A

2025-06-11 14:37:27,913 - INFO - Exported detailed trades to data/results/trades_detailed_XAUUSD_optimal_f.csv
2025-06-11 14:37:27,972 - INFO - Saved backtest results to data/results/backtest_results_XAUUSD_optimal_f.txt



=== XAUUSD Backtest Results ===
Bet Sizing Strategy: optimalf

Period: 2023-06-08 to 2024-11-22

ASIAN Session Performance:
Initial Capital: $100,000.00
Final Capital: $99,820.38
Total PnL: $-179.62
Return: -0.18%
Win Rate: 51.93% (148W/137L)
Max Drawdown: 0.94%
Sharpe Ratio: -0.20
Skewness: -0.088
Excess Kurtosis: 19.954

Attempt Analysis:

  Attempt 1:
  Trades: 249 (87.4% of session trades)
  PnL: $-111.84
  Win Rate: 53.82% (134W/115L)
  Average Win: $17.59
  Average Loss: $-21.66

  Attempt 2:
  Trades: 32 (11.2% of session trades)
  PnL: $-55.59
  Win Rate: 43.75% (14W/18L)
  Average Win: $1.99
  Average Loss: $-4.64

  Attempt 3:
  Trades: 4 (1.4% of session trades)
  PnL: $-12.19
  Win Rate: 0.00% (0W/4L)
  Average Win: $0.00
  Average Loss: $-3.05

LONDON Session Performance:
Initial Capital: $100,000.00
Final Capital: $99,669.46
Total PnL: $-330.54
Return: -0.33%
Win Rate: 49.74% (190W/192L)
Max Drawdown: 0.84%
Sharpe Ratio: -0.32
Skewness: -0.719
Excess Kurtosis: 18.520

At